In [34]:
import os
import random
import shutil
import sys

libri_path = "/home/gpadmin/tts/TensorFlowTTS/LibriTTS" # absolute path to TensorFlowTTS.
dataset_path = "/home/gpadmin/tts/TensorFlowTTS/libritts" # Change to your paths. This is a output of re-format dataset.
subset = "train-clean-360"

print(dataset_path)

/home/gpadmin/tts/TensorFlowTTS/libritts


In [35]:
with open(os.path.join(libri_path, "SPEAKERS.txt")) as f:
    data = f.readlines()

dataset_info = {}
max_speakers = 120  # Max number of speakers to train on
min_len = 5  # Min len of speaker narration time
max_file_len = 15  # max audio file lenght
min_file_len = 5  # min audio file lenght

In [36]:
possible_dataset = [i.split("|") for i in data[12:] if i.split("|")[2].strip() == subset and float(i.split("|")[3].strip()) >= min_len]

In [37]:
ids = [i[0].strip() for i in possible_dataset]

In [38]:
# pip install pysoundfile
import soundfile as sf

In [39]:
possible_map = {}
subset_path = os.path.join(libri_path, subset)
for i in os.listdir(subset_path):
    if i in ids:
        id_path = os.path.join(subset_path, i)
        id_dur = 0
        id_included = []

        for k in os.listdir(id_path):
            for j in os.listdir(os.path.join(id_path, k)):
                if ".wav" in j:
                    f_path = os.path.join(id_path, k, j)
                    sf_file =  sf.SoundFile(f_path)
                    dur = len(sf_file) / sf_file.samplerate
                    if max_file_len < dur or dur < min_file_len:
                        continue
                    else:
                        id_included.append(f_path)
                        id_dur += dur
        possible_map[i] = {"dur": id_dur, "included": id_included}


In [40]:
poss_speakers = {k: v["included"] for k, v in possible_map.items() if v["dur"]/60 >= min_len}

In [41]:
to_move = list(poss_speakers.keys())
random.shuffle(to_move)
to_move = to_move[:max_speakers]

In [42]:
for sp_id, v in poss_speakers.items():
    if sp_id in to_move:
        for j in v:
            f_name = j.split(os.path.sep)[-1]
            text_f_name = f_name.split(".wav")[0] + ".txt"
            os.makedirs(os.path.join(dataset_path, sp_id), exist_ok=True)
            shutil.copy(j, os.path.join(dataset_path, sp_id, f_name))
            shutil.copy(j.replace(".wav", ".normalized.txt"), os.path.join(dataset_path, sp_id, text_f_name))

In [44]:
import json

file_path = "/home/gpadmin/tts/TensorFlowTTS/libritts/libritts_mapper.json"
# Load the JSON file
with open(file_path, 'r') as json_file:
    data = json.load(json_file)

# Access the speakers_map and count its items
speakers_map = data.get('speakers_map', {})  # Default to an empty dictionary if 'speakers_map' is missing
number_of_items = len(speakers_map)

print(f'The "speakers_map" contains {number_of_items} items.')

The "speakers_map" contains 120 items.


In [ ]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices())